In [5]:
# Install packages
import os
from dotenv import load_dotenv
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, insert
from sqlalchemy.orm import Session

In [2]:
# load .env
load_dotenv()

# Create db connection string
db_username = os.getenv("DB_USERNAME")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_conn_str = f"mysql+pymysql://{db_username}:{db_password}@{db_host}:{db_port}/test_db"

# Set name for table in database
company_table = "company"

# Create database engine
db_engine = create_engine(db_conn_str)

# Load metadata container
metadata = MetaData()

# Create test insertion data
df_test = pd.DataFrame({"company_n":["Brookfield", "Telus"], "date_created": ["2025-12-06"]*2})

In [10]:
# Insert rows for new companies into company table

# First, define table using Table class
company_table_db = Table(company_table, metadata, autoload_with=db_engine)

# Create empty list for ids
ids = []

# Create session, and store ids
with Session(db_engine) as session:
    for _, row in df_test.iterrows():
        stmt = insert(company_table_db).values({"initial_name":row["company_n"], "date_created": row["date_created"]})
        result = session.execute(stmt)
        session.commit()
        ids.append(result.inserted_primary_key[0])

# Add the ids to the dataframe
df_test["company_id"] = ids

In [12]:
# Print dataframe to check if ids are as expected
df_test
# Output is as expected

,company_n,date_created,company_id
0,Brookfield,2025-12-06,3
1,Telus,2025-12-06,4
